# Assignment: Video Quality Inference

To this point in the class, you have learned various techniques for leading and analyzing packet captures of various types, generating features from those packet captures, and training and evaluating models using those features.

In this assignment, you will put all of this together, using a network traffic trace to train a model to automatically infer video quality of experience from a labeled traffic trace.

## Part 1: Warmup

The first part of this assignment builds directly on the hands-on activities but extends them slightly.

### Extract Features from the Network Traffic

Load the `netflix.pcap` file, which is a packet trace that includes network traffic. 

Click [here](https://github.com/noise-lab/ml-systems/blob/main/docs/notebooks/data/netflix.pcap) to download `netflix.pcap`.


In [2]:
import pandas as pd

In [3]:
# I opened wireshark and downloaded the pcap as a csv.
# That is why the filename below is different from what you may expect

packets = pd.read_csv("ml-systems-netflix-packets.csv")

In [4]:
packets.head(10)

,No.,Time,Source,Destination,Protocol,Length,Info,dest (r),dest(u),src (r),src(u),New Column
0,1,0.000000,192.168.43.72,128.93.77.234,DNS,77,Standard query 0xed0c A fonts.gstatic.com,53.0,53.0,55697.0,55697.0,1
1,2,0.000150,192.168.43.72,128.93.77.234,DNS,77,Standard query 0x301a AAAA fonts.gstatic.com,53.0,53.0,59884.0,59884.0,2
2,3,0.004726,192.168.43.72,128.93.77.234,DNS,87,Standard query 0x11d3 A googleads.g.doubleclic...,53.0,53.0,61223.0,61223.0,3
3,4,0.006522,192.168.43.72,128.93.77.234,DNS,87,Standard query 0x1284 AAAA googleads.g.doublec...,53.0,53.0,58785.0,58785.0,4
4,5,0.011103,192.168.43.72,128.93.77.234,DNS,78,Standard query 0x3432 AAAA ytimg.l.google.com,53.0,53.0,51938.0,51938.0,5
5,6,0.012354,192.168.43.72,128.93.77.234,DNS,96,Standard query 0xb756 A r4---sn-gxo5uxg-jqbe.g...,53.0,53.0,20949.0,20949.0,6
6,7,0.012474,192.168.43.72,128.93.77.234,DNS,75,Standard query 0x62ab A ssl.gstatic.com,53.0,53.0,58025.0,58025.0,7
7,8,0.012567,192.168.43.72,128.93.77.234,DNS,74,Standard query 0x42fb A www.google.com,53.0,53.0,15895.0,15895.0,8
8,9,0.319268,128.93.77.234,192.168.43.72,DNS,386,Standard query response 0x11d3 A 216.58.213.162,61223.0,61223.0,53.0,53.0,9
9,10,0.319288,192.168.43.72,128.93.77.234,DNS,75,Standard query 0x8756 A www.gstatic.com,53.0,53.0,18154.0,18154.0,10


### Cleaning the data frame

Before gettingstarted on the next stemp, I wanted to get a little more familiar with the data. First I wanted to answer:

1. What is the difference between "dest (r)" and "dest(u)"?
2. What is the difference between "src (r)" and "src(u)"?
3. What is going on with the "New Column" column?

I could of course look it up or ask Claude, but where is the fun in that? I wanted to first investigate what is going on myself.

In [12]:
# this cell shows there are 80 instances where "dest (r)" is not equal to "dest(u)"
len(packets) - sum(packets["dest (r)"] == packets["dest(u)"]) 

80

In [18]:
# we investigate what is happening in the cases where these aren't equal
packets[packets["dest (r)"] != packets["dest(u)"]].head(5)

,No.,Time,Source,Destination,Protocol,Length,Info,dest (r),dest(u),src (r),src(u),New Column
877,878,8.391182,CeLink_0d:2b:a7,Apple_01:4c:54,ARP,60,Who has 192.168.43.72? Tell 192.168.43.1,NaN,NaN,NaN,NaN,878
878,879,8.391235,Apple_01:4c:54,CeLink_0d:2b:a7,ARP,42,192.168.43.72 is at e4:ce:8f:01:4c:54,NaN,NaN,NaN,NaN,879
2324,2325,33.085201,fe80::a021:b7ff:febb:19ec,ff02::1,ICMPv6,86,Multicast Listener Query,NaN,NaN,NaN,NaN,2325
2388,2389,34.215829,fe80::e6ce:8fff:fe01:4c54,ff02::1:ff01:4c54,ICMPv6,86,Multicast Listener Report,NaN,NaN,NaN,NaN,2389
2459,2460,35.215949,fe80::e6ce:8fff:fe01:4c54,ff02::1:ff77:3595,ICMPv6,86,Multicast Listener Report,NaN,NaN,NaN,NaN,2460


In [24]:
# we can see that these 80 values are accounted for by NaN values
print(packets[~packets["dest (r)"].isna() & (packets["dest (r)"] != packets["dest(u)"])])

# I used Claude on this cell to debug an issue that I was having with indexing in pandas:
# https://claude.ai/share/d68d6049-2976-400d-9395-e8a6c1cf7f19

Empty DataFrame
Columns: [No., Time, Source, Destination, Protocol, Length, Info, dest (r), dest(u), src (r), src(u), New Column]
Index: []


In [46]:
# we can now show the same thing with "src (r)" and "src(u)"
# we can see that these 80 values are accounted for by NaN values
print(packets[~packets["src (r)"].isna() & (packets["src (r)"] != packets["src(u)"])])

KeyError: 'src (r)'

In [28]:
# now we investigate the "New Column"
new_col_list = packets["New Column"].to_list()

In [30]:
# now we show that it is just the row number indexed at one
for i in range(len(new_col_list)):
    assert i +1 == new_col_list[i]

So to answer my original questions from above:

1. The values for "dest (r)" and "dest(u)" are always identical
2. The values for "src (r)" and "src(u)" are always identical
3. The values in "New Column" contain no meaningful information.

Therefore we can colapse "dest (r)" and "dest(u)" into a single "dest" column. Likewise, we can colapse "src (r)" and "src(u)" into a single "src" column. We can drop "New Column" entirely.

In [51]:
# packets = packets.drop(['src (r)', 'dest (r)', 'New Column'], axis=1)
packets = packets.rename(columns={"dest(u)": "dest", "src(u)": "src"})

# I referenced the link below for how to drop a column in pandas
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html

# I referenced the link below to know how to rename columns in pandas
# https://stackoverflow.com/questions/11346283/renaming-column-names-in-pandas

In [52]:
packets

,No.,Time,Source,Destination,Protocol,Length,Info,dest,src
0,1,0.000000,192.168.43.72,128.93.77.234,DNS,77,Standard query 0xed0c A fonts.gstatic.com,53.0,55697.0
1,2,0.000150,192.168.43.72,128.93.77.234,DNS,77,Standard query 0x301a AAAA fonts.gstatic.com,53.0,59884.0
2,3,0.004726,192.168.43.72,128.93.77.234,DNS,87,Standard query 0x11d3 A googleads.g.doubleclic...,53.0,61223.0
3,4,0.006522,192.168.43.72,128.93.77.234,DNS,87,Standard query 0x1284 AAAA googleads.g.doublec...,53.0,58785.0
4,5,0.011103,192.168.43.72,128.93.77.234,DNS,78,Standard query 0x3432 AAAA ytimg.l.google.com,53.0,51938.0
...,...,...,...,...,...,...,...,...,...
141466,141467,495.764425,192.168.43.72,52.208.128.101,TCP,54,58518 > 443 [RST] Seq=28946 Win=0 Len=0,443.0,58518.0
141467,141468,495.764450,192.168.43.72,52.208.128.101,TCP,54,58518 > 443 [RST] Seq=28946 Win=0 Len=0,443.0,58518.0
141468,141469,495.764454,192.168.43.72,104.31.113.215,TCP,54,58530 > 80 [ACK] Seq=360 Ack=568 Win=262144 ...,80.0,58530.0
141469,141470,495.778858,172.217.18.195,192.168.43.72,TCP,66,"443 > 58514 [FIN, ACK] Seq=5175 Ack=3726 Win...",58514.0,443.0


### Identifying the Service Type

Use the DNS traffic to filter the packet trace for Netflix traffic.

### Generate Statistics

Generate statistics and features for the Netflix traffic flows. Use the `netml` library or any other technique that you choose to generate a set of features that you think would be good features for your model. 

**Write a brief justification for the features that you have chosen.**

### Inferring Segment downloads

In addition to the features that you could generate using the `netml` library or similar, add to your feature vector a "segment downloads rate" feature, which indicates the number of video segments downloaded for a given time window.

Note: If you are using the `netml` library, generating features with `SAMP` style options may be useful, as this option gives you time windows, and you can then simply add the segment download rate to that existing dataframe.

## Part 2: Video Quality Inference

You will now load the complete video dataset from a previous study to train and test models based on these features to automatically infer the quality of a streaming video flow.

For this part of the assignment, you will need two pickle files, which we provide for you by running the code below:

```

!gdown 'https://drive.google.com/uc?id=1N-Cf4dJ3fpak_AWgO05Fopq_XPYLVqdS' -O netflix_session.pkl
!gdown 'https://drive.google.com/uc?id=1PHvEID7My6VZXZveCpQYy3lMo9RvMNTI' -O video_dataset.pkl

```

### Load the File

Load the video dataset pickle file.

### Clean the File

1. The dataset contains video resolutions that are not valid. Remove entries in the dataset that do not contain a valid video resolution. Valid resolutions are 240, 360, 480, 720, 1080.

2. The file also contains columns that are unnecessary (in fact, unhelpful!) for performing predictions. Identify those columns, and remove them.

**Briefly explain why you removed those columns.**

### Prepare Your Data

Prepare your data matrix, determine your features and labels, and perform a train-test split on your data.

### Train and Tune Your Model

1. Select a model of your choice.
2. Train the model using your training data.

### Tune Your Model

Perform hyperparameter tuning to find optimal parameters for your model.

### Evaluate Your Model

Evaluate your model accuracy according to the following metrics:

1. Accuracy
2. F1 Score
3. Confusion Matrix
4. ROC/AUC

## Part 3: Predict the Ongoing Resolution of a Real Netflix Session

Now that you have your model, it's time to put it in practice!

Use a preprocessed Netflix video session to infer **and plot** the resolution at 10-second time intervals.